In [1]:
%pylab inline
import os.path
import jax
import jax.numpy as np
import jax_cosmo as jc

from astropy.io import fits
import h5py
import qp

Populating the interactive namespace from numpy and matplotlib


In [2]:
nz_shear = {}

with h5py.File("nz_qp_shear.h5", "r") as f:
    Z_MID = f['Z_MID'][::]
    nz_shear['0'] = f['nz_0'][::]
    nz_shear['1'] = f['nz_1'][::]
    nz_shear['2'] = f['nz_2'][::]
    nz_shear['3'] = f['nz_3'][::]
    nz_shear['4'] = f['nz_4'][::]

In [3]:
# First, let's define a function to go to and from a 1d parameter vector
def get_params_vec(cosmo, m, dz, ia):
    m1, m2, m3, m4 = m
    dz1, dz2, dz3, dz4 = dz
    A, eta = ia
    return np.array([ 
        # Cosmological parameters
        cosmo.sigma8, cosmo.Omega_c, cosmo.Omega_b,
        cosmo.h, cosmo.n_s, cosmo.w0,
        # Shear systematics
        m1, m2, m3, m4,
        # Photoz systematics
        dz1, dz2, dz3, dz4,
        # IA model
        A, eta
    ])
    
def unpack_params_vec(params):
    # Retrieve cosmology
    cosmo = jc.Cosmology(sigma8=params[0], Omega_c=params[1], Omega_b=params[2],
                         h=params[3], n_s=params[4], w0=params[5],
                         Omega_k=0., wa=0.)
    m1,m2,m3,m4 = params[6:10]
    dz1,dz2,dz3,dz4 = params[10:14]
    A = params[14]
    eta = params[15]
    return cosmo, [m1,m2,m3,m4], [dz1,dz2,dz3,dz4], [A, eta]

# Let's try a round trip just to make sure
p = get_params_vec(jc.Planck15(), [1.,2.,3.,4.], [5.,6.,7.,8.],
              [1., 2.])
unpack_params_vec(p)

(Cosmological parameters: 
     h:        0.6774 
     Omega_b:  0.0486 
     Omega_c:  0.2589 
     Omega_k:  0.0 
     w0:       -1.0 
     wa:       0.0 
     n:        0.9667 
     sigma8:   0.8159,
 [DeviceArray(1., dtype=float32),
  DeviceArray(2., dtype=float32),
  DeviceArray(3., dtype=float32),
  DeviceArray(4., dtype=float32)],
 [DeviceArray(5., dtype=float32),
  DeviceArray(6., dtype=float32),
  DeviceArray(7., dtype=float32),
  DeviceArray(8., dtype=float32)],
 [DeviceArray(1., dtype=float32), DeviceArray(2., dtype=float32)])

In [4]:
# Unpack parameter vector
cosmo, m, dz, (A, eta) = unpack_params_vec(p) 
bw = 0.01
gals_per_arcmin2 = 10
neff_s = [1.47, 1.46, 1.50, 0.73]

In [5]:
def apply_shift(z, nz_inp):
    nz = jc.redshift.kde_nz(z,  nz_inp, bw = bw,  gals_per_arcmin2 = gals_per_arcmin2)
    nzs_s = [jc.redshift.kde_nz(z,
                            nz, 
                            bw = bw,
                            gals_per_arcmin2=neff_s[i - 1])
           for i in range(1, 5)]
    nzs_s_sys = [jc.redshift.systematic_shift(nzi, dzi) for nzi, dzi in zip(nzs_s, dz)]
    return nzs_s_sys

In [6]:
nz_sys_shear = {}

for i in range(1, 5):
    shifted_nz = apply_shift(Z_MID, nz_shear[str(i)])[i-1]
    nz_sys_shear[str(i)] = shifted_nz